In [37]:
from openai import OpenAI
from datetime import datetime
from llmfactor import LLMFactorAnalyzer, LoggerSingleton
from pandas import Timestamp

In [38]:
client = OpenAI(
    base_url="http://localhost:8080/v1",
    api_key="token-abc123",
)

In [39]:
analyzer = LLMFactorAnalyzer(
    base_url="http://localhost:8080/v1",
    api_key="token-abc123",
    model="./models/llama-3.1-8B-instruct-Q8_0.gguf",
    logger=LoggerSingleton.get_logger())

In [40]:
tickers = analyzer.get_available_tickers()
ticker = tickers[0]
stock_target = ticker

factor_k = 5
price_k = 5

dates = analyzer.get_available_dates(ticker, price_k)
target_date = dates[1]

news_str = analyzer.news_data.get_news_by_date(ticker, target_date)
price_movements = analyzer.price_data.get_price_movements(ticker, target_date, price_k)
price_str, price_str_last = analyzer.format_price_movements(price_movements, ticker, target_date)
print(price_str + price_str_last)
print(*price_movements, sep="\n")

On 2018-01-04, the stock price of AAPL rose.
On 2018-01-05, the stock price of AAPL rose.
On 2018-01-08, the stock price of AAPL fell.
On 2018-01-09, the stock price of AAPL fell.
On 2018-01-10, the stock price of AAPL
{'date': Timestamp('2018-01-04 00:00:00'), 'rise': True}
{'date': Timestamp('2018-01-05 00:00:00'), 'rise': True}
{'date': Timestamp('2018-01-08 00:00:00'), 'rise': False}
{'date': Timestamp('2018-01-09 00:00:00'), 'rise': False}
{'date': Timestamp('2018-01-10 00:00:00'), 'rise': False}


In [41]:
factor_extraction = client.chat.completions.create(
    model=analyzer.model,
    temperature=0,
    max_tokens=factor_k * 200,
    messages=[
        {"role": "system", "content": f"Please extract the top {factor_k} factors that may affect the stock price of {ticker} from the following news."},
        {"role": "user", "content": news_str},
        {"role": "assistant", "content": f"The following are the main {factor_k} factors that may affect {ticker}’s stock price recently:"}
    ])

factor_str = factor_extraction.choices[0].message.content
print(factor_str)

Based on the text, the main 5 factors that may affect AAPL's stock price recently are:

1. **Shazam Acquisition**: Apple's acquisition of Shazam, a music recognition app, may impact its services business and revenue growth.
2. **iPhone Slowdown Controversy**: Apple's admission of deliberately slowing down older iPhones to prevent battery issues has sparked controversy and raised questions about the company's transparency.
3. **Jimmy Iovine's Departure Rumors**: Rumors that Jimmy Iovine, Apple Music's executive, plans to leave the company have been denied by Iovine himself, but may still impact investor confidence.
4. **Qualcomm's Semiconductor Market Share**: Qualcomm's challenges in its technology licensing business may impact Apple's reliance on the company's semiconductor operations.
5. **Repatriation of Cash**: Apple's plans for its repatriated cash, which may be used for share buybacks, dividends, or other investments, may impact its stock price.

These factors may have a signific

In [42]:
# factor_str = analyzer.opposite_meaning_factors(factor_str)
# print(factor_str)

In [44]:
answer_extraction = client.chat.completions.create(
    model=analyzer.model,
    temperature=0,
    messages=[
        {"role": "system", "content": "Based on the following information, please judge the direction of the stock price from rise/fall, fill in the blank and give reasons."},
        {"role": "system", "content": f"These are the main factors that may affect this stock’s price recently: {factor_str}."},
        {"role": "system", "content": price_str},
        {"role": "assistant", "content": price_str_last},
    ]
)

answer = answer_extraction.choices[0].message.content
print(answer)

Based on the information provided, I would predict that the stock price of AAPL on 2018-01-10 will **rise**.

My reasoning is as follows:

* The controversy surrounding the iPhone slowdown has already been reported and may have had its impact on the stock price.
* The departure rumors of Jimmy Iovine have been denied, which may have alleviated some concerns.
* Qualcomm's challenges in its technology licensing business may have a longer-term impact, but it's unclear how this will affect Apple's stock price in the short term.
* The repatriation of cash is a positive development for Apple, which may lead to increased investor confidence and a rise in the stock price.

Overall, while there are still some uncertainties surrounding Apple's stock price, the positive news about the repatriation of cash may outweigh the negative factors, leading to a rise in the stock price on 2018-01-10.
